# Import the needed libraries ( hipercube class and hypercubes)

In [ ]:
from hypercube import Hypercube
import os
import glob
import numpy as np
from squares import SquareDetector
from segment_anything_meta import Segmenter


# Initialize hypercubes

In [ ]:
# @title Add your own values { run: "auto" }

path = "C:\\Users\\Fran\\Downloads\\Hypercubos\\Hypercubos\\ABS" # @param {type:"string"}
hypercube = Hypercube()

all_files = glob.glob(os.path.join(path , "*.bin"))
print(len(all_files))

# Preprocess
Here we will make use of the preprocess library in order to apply different filters and test the behavior of the net in different circumstances. 
In order to make use of this library we should know what every filter is used for and apply it proper, for that pourpose I recommend to have a look to the library documentation.


### pca

In [ ]:

import sys
sys.path.insert(1, r'C:\Users\Fran\Projects\Tfg_IA\Preprocess')
import preprocess as preprocess
import matplotlib.pyplot as plt
ncomponents = 3
hipercubo = Hypercube()
hipercubo.Load(all_files[0])
hipercubo.Crop(94,528,10,214)
hipercubo.average()
print(hipercubo.hipercubo.shape)
pc_img = preprocess.pca(hipercubo,100,ncomponents)
print(pc_img.shape)
for i in range(ncomponents):
    plt.title(f'PC - {i}')
    plt.imshow(pc_img[i,:,:], cmap='nipy_spectral')
    plt.axis('off')
    plt.show()

### SNV

In [ ]:

import sys
sys.path.insert(1, r'C:\Users\Fran\Projects\Tfg_IA\Preprocess')
import preprocess as preprocess
import matplotlib.pyplot as plt
hipercubo = Hypercube()
hipercubo.Load(all_files[0])
hipercubo.Crop(94,528,10,214)
hipercubo.average()
hipercubo.PlotIMGBidimensional()
print(hipercubo.hipercubo.max(),hipercubo.hipercubo.min())
# hipercubo.plotspectres()

# Plot SNV
standardNormalizated = preprocess.standardNormalizationVariate(hipercubo)
for i in range(len(standardNormalizated)):
    plt.plot(standardNormalizated[i])
plt.show()  


### Compare different point spectres

In [ ]:

import sys
sys.path.insert(1, r'C:\Users\Fran\Projects\Tfg_IA\Preprocess')
import preprocess as preprocess
import matplotlib.pyplot as plt
from ponit_selector import PointSelector
hipercubo = Hypercube()
hipercubo.Load(all_files[0])
hipercubo.Crop(94,528,10,214)
hipercubo.average()
hipercubo.PlotIMGBidimensional()
# hipercubo.plotspectres()

# Select Roi from hipercubo.hipercubo and plot the selected points
image = hipercubo.returnImage()
selector = PointSelector(image, "Selecciona los puntos")
selector.show_and_wait()
for i in range(len(selector.points)):
    plt.plot(hipercubo.hipercubo[selector.points[i][1],:,selector.points[i][0]])
plt.show()
selector = PointSelector(image, "Selecciona los puntos")
selector.show_and_wait()
for i in range(len(selector.points)):
    plt.plot(hipercubo.hipercubo[selector.points[i][1],:,selector.points[i][0]])
plt.show()

# Find max and min values in the spectres to find while filtering

In [ ]:
maxValue = 0
minValue = 1000
for filename in all_files:
    hipercubo = Hypercube()
    hipercubo.Load(filename)
    hipercubo.median()
    if maxValue < np.max(hipercubo.medianHipercube):
        maxValue = np.max(hipercubo.medianHipercube)
    if minValue > np.min(hipercubo.medianHipercube): 
        minValue = np.min(hipercubo.medianHipercube)

## First way to preprocess ( Obtaining each pixel classified)

In [ ]:
# @title Add your own values { display-mode: "form" }
y1 = 94 # @param {type:"integer"}
y2 = 528 # @param {type:"integer"}
x1 = 10 # @param {type:"integer"}
x2 = 214 # @param {type:"integer"}
annotation = "pp_pe" # @param ["pp_pe", "abs", "pet"] {allow-input: true}

for filename in all_files:
    hipercubo = Hypercube()
    hipercubo.Load(filename)
    hipercubo.Crop(y1,y2,x1,x2)
    # hipercubo.PlotIMG(100)
    hipercubo.average()
    # hipercubo.PlotDimLine(100)
    # hipercubo.PlotIMG(100)
    # hipercubo.PlotIMGBidimensional()
    # hipercubo.selectROI()
    # hipercubo.PlotIMG(100)
    segment = Segmenter("vit_h",r"sam_vit_h_4b8939.pth")
    image = hipercubo.returnImage()
    mask = segment.segmentRoi(image)
    finalmask,maskLabeled = hipercubo.extractValues(mask)
    hipercubo.saveValues(finalmask, annotation)
    hipercubo.saveMask(maskLabeled,filename)
    print("Done")

## Second way to process (obtaining the hole ibage labeled)

In [ ]:
counter = 0
for filename in all_files:
    hipercubo = Hypercube()
    hipercubo.Load(filename)
    hipercubo.Crop(94,528,10,214)
    # hipercubo.PlotIMG(100)
    hipercubo.average()
    # hipercubo.PlotDimLine(100)
    # hipercubo.PlotIMG(100)
    # hipercubo.PlotIMGBidimensional()
    # hipercubo.selectROI()
    # hipercubo.PlotIMG(100)
    segment = SquareDetector(200)
    image = hipercubo.returnImage()
    mask = segment.selectSquares(image)
    finalmask,maskLabeled = hipercubo.extractSquares(mask)
    for i in range(len(finalmask)):
        hipercubo.saveValues(finalmask[i], r"C:\Users\Fran Quiles\Documents\Projects\Tfg_IA\Classifiers\CNN\SPLIT\pp_pe"+ str(counter) + str(i))
    print("Done")
    counter += 1

## Third way to label 
Here the idea is to make use of external applications that already exist in the market. What we want to do is to take profit of the Saas that we already have in order to annotate images. I have noticed that what we are really anotating is an image that we preprocess to have a better comprenhension about what the data looks like. What we are going to do here is:
- Charge the files
- preprocess the images
    - Spatial crop
    - Average
    - Some preprocess to improve the image view to the human being
- Export image to an image format

Once we have the image preprocess to be process by a human what we have to do is to send the image to our labeling machine, we have a bunch of labeling softawares:
- AWS
- Microsoft azure
- Roboflow
- CVAT
- Label Box
- Scale AI
- Make Sense
- VGG Image Annotator
- Labelme
- Dashdoodler
- Label Studio

There are some of them that we can simple run in a docker image locally, so depending on what tool we prefer we should send the images in one or another way.

Once we have our images labeleds depending on what is the format labeled we have to create a  converter to match this information with our data, depending on what we really want.